In [ ]:
import hydra
import matplotlib.pyplot as plt
import numpy as np
from omegaconf import OmegaConf


In [ ]:
from mantis.configs.config import Config

with hydra.initialize(version_base=None, config_path="../conf/"):
    cfg = hydra.compose(config_name="config")
    OmegaConf.resolve(cfg)
    config = Config(**cfg)

In [ ]:
from mantis.data.datasets import setup_dataset

train_ds, val_ds = setup_dataset(config.dataset)

In [ ]:
from torch.utils.data import DataLoader

B = 2

loader = DataLoader(train_ds, batch_size=B, shuffle=True)

In [ ]:
import torch

from mantis.model.patch_extractor import extract_equal_zoom_patches, extract_patches


def vis_patches(batch: dict[str, torch.Tensor], z: float):
    P = 16
    imgs = (batch["image"] * 255).round().int().clip(0, 255)
    imgs = imgs.permute(0, 2, 3, 1).numpy().astype(np.uint8)
    locs = torch.rand((B, 2))
    scaled_locs = locs * 2 - 1
    patches = extract_equal_zoom_patches(batch["image"], locs=scaled_locs, z=z, patch_size=P)
    inp_dim = min(config.dataset.img_height, config.dataset.img_width)
    size = inp_dim / (2 ** z)
    print(f"Size: {size}")

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(imgs[0])
    axs[0].set_axis_off()
    axs[0].scatter(locs[0, 0] * config.dataset.img_width, locs[0, 1] * config.dataset.img_height, c="red")
    min_loc = (locs[0, 0] * config.dataset.img_width - size / 2).item(), (locs[0, 1] * config.dataset.img_height - size / 2).item()
    print(min_loc)
    print(size)
    axs[0].add_patch(
        plt.Rectangle(min_loc, size, size, fill=False, edgecolor="red", linewidth=2)
    )

    axs[1].imshow(imgs[1])
    axs[1].set_axis_off()
    axs[1].scatter(locs[1, 0] * config.dataset.img_width, locs[1, 1] * config.dataset.img_height, c="red")
    min_loc = (locs[1, 0] * config.dataset.img_width - size / 2).item(), (locs[1, 1] * config.dataset.img_height - size / 2).item()
    axs[1].add_patch(
        plt.Rectangle(min_loc, size, size, fill=False, edgecolor="red", linewidth=2)
    )
    plt.show()

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    patch = (patches[0] * 255).round().int().permute(1, 2, 0).numpy().astype(np.uint8)
    axs[0].imshow(patch)
    axs[0].set_axis_off()

    patch = (patches[1] * 255).round().int().permute(1, 2, 0).numpy().astype(np.uint8)
    axs[1].imshow(patch)
    axs[1].set_axis_off()

    plt.show()


def vis_random_patches(ind: int) -> None:
    print(f"Index: {ind}")
    batch = train_ds[ind]
    img = batch["image"]
    img_vis = (img * 255).round().int().clip(0, 255)
    img_vis = img_vis.permute(1, 2, 0).numpy().astype(np.uint8)

    # Sample locations
    M = 1
    max_z = 0
    locs = torch.zeros((1, M, 3))
    locs[:, :, :2] = torch.rand((1, M, 2))
    locs[:, :, 2] = torch.rand((1, M)) * max_z
    locs[:, :, 2] = 3.2288
    min_dim = min(config.dataset.img_height, config.dataset.img_width)


    plt.imshow(img_vis)
    plt.axis("off")
    plt.scatter(locs[0, :, 1] * config.dataset.img_width, locs[0, :, 0] * config.dataset.img_height, c="red")
    for i in range(M):
        y, x, z = locs[0, i]
        x = int(x * config.dataset.img_width)
        y = int(y * config.dataset.img_height)
        size = min_dim / (2 ** z)
        size = size.clamp(1, min_dim)
        size = size.item()
        min_x = x - size // 2
        min_y = y - size // 2
        plt.gca().add_patch(
            plt.Rectangle((min_x, min_y), size, size, fill=False, edgecolor="red", linewidth=2)
        )
    plt.show()

    P = 32
    patches = extract_patches(img.unsqueeze(0), locs, P).squeeze(0)
    for i in range(M):
        patch = (patches[i] * 255).round().int().permute(1, 2, 0).numpy().astype(np.uint8)
        plt.imshow(patch)
        plt.axis("off")
        plt.show()

In [ ]:
batch = next(iter(loader))
vis_patches(batch, z=4)

In [ ]:
import random

ind = random.randint(0, len(train_ds))
ind = 16586
vis_random_patches(ind)